# Experimental Results 2: Building Distribution B

After running some initial experiments on a single environment, let us consider multiple buildings at the same time.

<div class="alert alert-info">

Note

In this notebook Gym v0.21 is used, because this version is required for Stable Baselines3 (SB3), an RL algorithm library used here. Thus, the environment API is different compared to other pages in the Bauwerk docs.

</div>

## Evaluation

As in previous experiments we consider the *performance relative to random and optimal control*,

$p' = \frac{p_m-p_r}{p_o-p_r}$,

where $p_m$ is the average reward of the method to be evaluated, $p_r$ and $p_o$ the average rewards of random and optimal control respectively. 

<div class="alert alert-info">

Note

With the performance measure $p'$, a value $>0$ means that the method is better than random, a value close to $1$ means that the method is close to optimal, a value $<0$ means that the method is worse than random, i.e. completely useless.

</div>



## Setting up distribution

To start off we import all relevant packages and sample tasks from distribution

In [ ]:
# Setup and helper code
import bauwerk
import bauwerk.evaluation
import bauwerk.benchmarks
import gym
import numpy as np

# The length of each task, i.e. how long we aim to 
TASK_LEN = 24*30 # evaluate on 1 month of actions

# Create SolarBatteryHouse environment
build_dist_b = bauwerk.benchmarks.BuildDistB(seed=100, task_ep_len=TASK_LEN)
print(f"Sampled {len(build_dist_b.train_tasks)} training and"
    f" {len(build_dist_b.test_tasks)} test tasks from building distribution B.")

## Baselines

### Random, optimal & *nocharge* performance

To get started, we compute the performance of taking random, optimal and no charging actions in
each environment. We can see that, with random actions, a larger battery size leads to worse performance as the battery size allows for more (expensive) charging from the grid. On the other hand, for optimal actions, increasing battery size leads to increasing performance as it provides additional flexibility. This effect plateaus eventually as larger battery sizes and additional flexibility can no longer be used within the system. As expected, when not taking any charging actions there is no effect of battery size on performance. Note that here we only consider *operational performance* but ignore other costs such as acquiring and installing a larger battery.

In [ ]:
# Get data on different training tasks
env = build_dist_b.make_env()
batt_sizes = []
perf_opts = []
perf_rands = []
perf_nocharges = []
for i, task in enumerate(build_dist_b.train_tasks):
    env = build_dist_b.make_env()
    env.set_task(task)
    perf_opt = bauwerk.evaluation.get_optimal_perf(env, TASK_LEN=TASK_LEN)
    perf_rand, perf_rand_std = bauwerk.evaluation.get_avg_rndm_perf(
        env, 
        TASK_LEN=TASK_LEN,
        num_samples=10,
    )
    perf_nocharge = bauwerk.evaluation.evaluate_actions(np.zeros((TASK_LEN,1)), env)
    batt_sizes.append(env.cfg.battery_size)
    perf_opts.append(perf_opt)
    perf_rands.append(perf_rand)
    perf_nocharges.append(perf_nocharge)

In [ ]:
# Plot the effect of battery size in training task performance
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
plt.subplot(1, 3, 2)
plt.scatter(batt_sizes, perf_opts)
plt.xlabel("Battery size (kWh)")
plt.title("Optimal actions")
plt.subplot(1, 3, 1)
plt.title("Random actions")
plt.ylabel("Avg energy payment per step ($)")
plt.scatter(batt_sizes, perf_rands)
plt.xlabel("Battery size (kWh)")
plt.subplot(1, 3, 3)
plt.title("No charging actions")
plt.scatter(batt_sizes, perf_nocharges)
plt.xlabel("Battery size (kWh)")
plt.tight_layout()

## RL agents

Next we consider simple reinforcement learning (RL) agents. We use [Stable Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) to access RL algorithm implementations. 

In [ ]:
# Helper functions for evaluating methods

from stable_baselines3.common.callbacks import BaseCallback

def eval_model(model, env):
    # Obtaining model actions and evaluating them
    model_actions = []
    obs = env.reset()
    for i in range(TASK_LEN):
        action, _states = model.predict(obs)
        model_actions.append(action)
        obs, _, _, _ = env.step(action)

    p_model = bauwerk.evaluation.evaluate_actions(model_actions[:TASK_LEN], env)
    return p_model

# callback for evaluating callback during training
class EvalCallback(BaseCallback):
    def __init__(self, eval_freq = 24*7, verbose=0):
        super().__init__(verbose)
        self.data = []
        self.eval_freq = eval_freq
        self.eval_env = gym.make("bauwerk/SolarBatteryHouse-v0")

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.data.append(eval_model(self.model, self.eval_env))

    def _on_step(self) -> bool:
        if self.num_timesteps % self.eval_freq == 0:
            self.data.append(eval_model(self.model, self.eval_env))

        return True

# Measuring performance relative to random and optimal
def compute_rel_perf(p_model, p_rand, p_opt):
    return (p_model - p_rand)/(p_opt - p_rand)

### SAC

After setting up all helper functions we consider a popular RL algorithm - *Soft Actor-Critic* (SAC).

In [ ]:
from stable_baselines3 import SAC

model_sac = SAC(
    policy="MultiInputPolicy",
    env="bauwerk/SolarBatteryHouse-v0", 
    verbose=0,
)
sac_callback = EvalCallback()
model_sac.learn(total_timesteps=NUM_TRAIN_STEP,callback=sac_callback)

p_model_sac = eval_model(model_sac, env)

print(f"Avg reward (per step) with model actions: {p_model_sac:.4f}")

p_rel_sac = compute_rel_perf(p_model_sac)
print(f"Performance relative to random and optimal: {p_rel_sac:.4f}")

Given the same amount of training time on the environment (1 year), SAC notably outperforms PPO.

## Results plot

Next we plot the combined results of the baselines, random and optimal actions, and our RL agents, PPO and SAC.

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(0,NUM_TRAIN_STEP,ppo_callback.eval_freq)
plt.plot(
    x, ppo_callback.data[:NUM_TRAIN_STEP//ppo_callback.eval_freq + 1], label="PPO"
)
plt.plot(
    x, sac_callback.data[:NUM_TRAIN_STEP//ppo_callback.eval_freq + 1], label="SAC"
)
plt.hlines(p_opt, 0, NUM_TRAIN_STEP, label="Optimal", linestyle=":", color="black")
plt.hlines(p_rand, 0, NUM_TRAIN_STEP, label="Random", linestyle="--", color="grey")
plt.hlines(p_nocharge, 0, NUM_TRAIN_STEP, label="No charging", linestyle="-.", color="lightblue")
plt.legend()
plt.ylabel(f"avg grid payment (per timestep)")
plt.xlabel(f"timesteps (each {env.cfg.time_step_len}h)")

> Note for reproducibility: this entire notebook was executed inside the devcontainer of the Bauwerk repository.